# Data cleaning of the 145 buildings dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
from pathlib import Path

In [ ]:
path_data = 'C:/Users/Orson/Documents/Digital_Lab/Projet_GAC/Datasets/'

## Importing data

In [ ]:
def dateparse (timestamp): 
    return datetime.datetime.strptime(timestamp,'%Y-%m-%dT%H:%M:%S%z')

In [33]:
data = pd.read_csv(path_data+'databat/daily_conso.csv',sep=',',
                        parse_dates=['timestamp'],
                        date_parser = dateparse)
data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(tzinfo=None))

In [34]:
data.head()

,id,departement,surface,quantite_personne,date_construction,type_chauff_premier,type_chauff_secondaire,type_chauff_auxiliaire,timestamp,consumption
0,1,85,160.0,6,entre 1971 et 1990,électrique,bois,bois,2015-11-30,26585.0
1,1,85,160.0,6,entre 1971 et 1990,électrique,bois,bois,2015-12-01,20115.0
2,1,85,160.0,6,entre 1971 et 1990,électrique,bois,bois,2015-12-02,30985.0
3,1,85,160.0,6,entre 1971 et 1990,électrique,bois,bois,2015-12-03,29985.0
4,1,85,160.0,6,entre 1971 et 1990,électrique,bois,bois,2015-12-04,21232.5


## Creating metadata file

In [36]:
metadata2000 = data[['id','surface']].groupby(['id','surface']).count().reset_index().rename(columns={'id':'bat_id'})

series_columns = pd.Series({'is_house':True,
                              'time_step': 1440,
                              'lat':np.nan,
                              'long':np.nan,
                              'monday_is_off':np.nan,
                              'tuesday_is_off':np.nan,
                              'wednesday_is_off':np.nan,
                              'thursday_is_off':np.nan,
                              'friday_is_off':np.nan,
                              'saturday_is_off':np.nan,
                              'sunday_is_off':np.nan})
for i in range(4):
    metadata2000.insert(i+1,series_columns.index[i],series_columns[i])
for i in range(4, 11):
    metadata2000.insert(i+2,series_columns.index[i],series_columns[i])

metadata2000['bat_id'] = pd.Series([2000+i for i in range(1,max(metadata2000['bat_id'])+1)])
metadata2000 = metadata2000.set_index('bat_id')    

In [37]:
metadata2000.to_csv(path_data+'databat/metadata2000.csv',sep=';')

## Creating each building consumption file

###  Finding temperatures

In [6]:
df_stations = pd.read_csv(path_data + 'stations.csv',sep=';')[['ID','Latitude','Longitude']]\
                    .rename(columns={'ID':'station_id','Latitude':'lat','Longitude':'long'})

In [7]:
df_stations.head()

,station_id,lat,long
0,7005,50.136000,1.834000
1,7015,50.570000,3.097500
2,7020,49.725167,-1.939833
3,7027,49.180000,-0.456167
4,7037,49.383000,1.181667


In [8]:
df_departements = pd.read_csv(path_data +'centres.csv',sep=',')

In [9]:
from lat_lon_parser import parse

df_centres = df_departements[['N° ','LONGITUDE ','LATITUDE ']]\
                .rename(columns={'N° ':'departement','LONGITUDE ':'long','LATITUDE ':'lat'})\
                .reindex(columns=['departement','lat','long'])\
                .set_index('departement')


df_centres['lat'] = df_centres['lat'].apply(lambda x: parse(x))
df_centres['long'] = df_centres['long'].apply(lambda x: parse(x))

In [10]:
df_centres.head()

,lat,long
departement,,
01,46.099444,5.348889
02,49.559444,3.558333
03,46.393611,3.188333
04,44.106111,6.243889
05,44.663611,6.263056


In [11]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [12]:
def min_distance(x):
    return df_stations['station_id'][\
                             np.argmin(\
                     [haversine(x['long'],x['lat'],df_stations['long'][i],df_stations['lat'][i])\
                      for i in range(len(df_stations))]
                                      )
                            ]

In [13]:
stations_series = df_centres.apply(lambda x: min_distance(x),axis=1)
df_stations_dep = pd.DataFrame({'departement':stations_series.index,'station_id':stations_series.values})
df_stations_dep = df_stations_dep.drop(\
                                       df_stations_dep[df_stations_dep['departement'] == '2A '].index)
df_stations_dep = df_stations_dep.drop(\
                                       df_stations_dep[df_stations_dep['departement'] == '2B '].index)
df_stations_dep['departement'] = df_stations_dep['departement'].apply(lambda x: int(x))
df_stations_dep = df_stations_dep.set_index('departement')

In [14]:
df = data[['id','departement','timestamp','consumption']].rename(columns={'id':'bat_id','consumption':'active_power'})
df = df.join(df_stations_dep,on='departement')

In [15]:
stations = df['station_id'].values

In [16]:
list_filenames = []
for filename in os.listdir(path_data+'temperature_datetimes'):
    list_filenames.append(filename)

In [17]:
def dateparse_2 (timestamp): 
    return datetime.datetime.strptime(timestamp,'%d/%m/%Y %H:%M:%S')

In [18]:
df_temperatures = pd.DataFrame(columns=['station_id','timestamp','temperature'])
for filename in list_filenames:
    df_temp = pd.read_csv(path_data+'temperature_datetimes/'+filename,sep=';',parse_dates=['date'],na_values='mq')
    df_temp = df_temp[df_temp['numer_sta'].isin(stations)][['numer_sta','date','t']]
    df_temp = df_temp.rename(columns={'numer_sta':'station_id','t':'temperature','date':'timestamp'})
    df_temperatures = df_temperatures.append(df_temp,ignore_index=True)

df_temperatures = df_temperatures.sort_values(by=['station_id','timestamp'])
df_temperatures['temperature'] = df_temperatures['temperature'].apply(lambda x: x-273.15)
df_temperatures['station_id'] = pd.to_numeric(df_temperatures['station_id'])

In [19]:
df_temperatures['timestamp'] = df_temperatures['timestamp'].apply(lambda x: datetime.datetime(year=x.year, month=x.month, day=x.day))

In [20]:
df_temperatures = df_temperatures.groupby(['station_id','timestamp']).mean().reset_index()

In [21]:
df_merge = pd.merge(df,df_temperatures,
                how='inner',
                left_on=['station_id','timestamp'],
                right_on=['station_id','timestamp'])
df_merge = df_merge.sort_values(['bat_id','timestamp'])

### Missing temperatures

In [22]:
df_merge[df_merge['temperature'].isnull()]

,bat_id,departement,timestamp,active_power,station_id,temperature
24436,16,2,2017-12-13,16400.0,7072,NaN
37117,40,11,2019-07-27,16903.0,7747,NaN
37118,40,11,2019-07-28,14450.0,7747,NaN
37139,40,11,2019-08-18,18697.0,7747,NaN
37540,42,83,2019-09-11,10675.0,7661,NaN
37583,42,83,2019-10-24,13117.0,7661,NaN


In [23]:
temp = pd.merge(df,df_temperatures,
                how='outer',
                left_on=['station_id','timestamp'],
                right_on=['station_id','timestamp'])

In [24]:
temp[temp['temperature'].isnull() == True].dropna(subset=['bat_id'])

,bat_id,departement,timestamp,active_power,station_id,temperature
8539,7.0,67.0,2016-08-15,21926.0,7190,NaN
24437,16.0,2.0,2017-12-13,16400.0,7072,NaN
37118,40.0,11.0,2019-07-27,16903.0,7747,NaN
37119,40.0,11.0,2019-07-28,14450.0,7747,NaN
37140,40.0,11.0,2019-08-18,18697.0,7747,NaN
37541,42.0,83.0,2019-09-11,10675.0,7661,NaN
37584,42.0,83.0,2019-10-24,13117.0,7661,NaN
48128,63.0,12.0,2019-07-09,12077.0,7558,NaN
48129,63.0,12.0,2019-07-10,11014.0,7558,NaN
48130,63.0,12.0,2019-07-11,11544.0,7558,NaN


### Saving files

In [27]:
df_final = df_merge[['bat_id','timestamp','active_power','temperature']]

In [28]:
df_final.head()

,bat_id,timestamp,active_power,temperature
0,1,2015-11-30,26585.0,11.6000
2,1,2015-12-01,20115.0,10.9500
4,1,2015-12-02,30985.0,5.2125
8,1,2015-12-03,29985.0,5.1750
12,1,2015-12-04,21232.5,8.4125


In [29]:
ids = np.unique(df_final['bat_id'])
for i in ids:
    df_bat = df_final[df_final['bat_id']==i][['timestamp','active_power','temperature']]
    if i < 10:
        df_bat.to_csv(path_data+'databat/200{}.csv'.format(i),sep=';',index=False)
    elif 10 <= i < 100:
        df_bat.to_csv(path_data+'databat/20{}.csv'.format(i),sep=';',index=False)
    else:
        df_bat.to_csv(path_data+'databat/2{}.csv'.format(i),sep=';',index=False)